<a href="https://colab.research.google.com/github/itsayushthada/FastAI2019/blob/master/Benchmark/CIFAR100_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>